# Password Dumper Remote Thread in LSASS
Detects password dumper activity by monitoring remote thread creation EventID 8 in combination with the lsass.exe process as TargetImage. The process in field Process is the malicious program. A single execution can lead to hundreds of events.

## Rule Content
```
- title: Password Dumper Remote Thread in LSASS
  id: f239b326-2f41-4d6b-9dfa-c846a60ef505
  description: Detects password dumper activity by monitoring remote thread creation
    EventID 8 in combination with the lsass.exe process as TargetImage. The process
    in field Process is the malicious program. A single execution can lead to hundreds
    of events.
  references:
  - https://jpcertcc.github.io/ToolAnalysisResultSheet/details/WCE.htm
  status: stable
  author: Thomas Patzke
  logsource:
    product: windows
    service: sysmon
    category: null
  detection:
    selection:
      EventID: 8
      TargetImage: C:\Windows\System32\lsass.exe
      StartModule: null
    condition: selection
  tags:
  - attack.credential_access
  - attack.t1003
  - attack.s0005
  falsepositives:
  - unknown
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-sysmon-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"8" AND target_process_path:"C\:\\Windows\\System32\\lsass.exe" AND NOT _exists_:thread_start_module)')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()